In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot') 
%matplotlib inline
# import cufflinks as cf
# import holoviews as hv
from numpy.random import *

In [3]:
seed(8492)

"""
サンプル生成用の関数
"""
def create_sample(n):
    #XとZを相関を持つように作る
    cX = multivariate_normal( [10,10],  [[10,8],[8,10]],  n)
    
    #誤差項
    eps = normal(0, 5, n)

    #観測不可能な交絡因子
    conf = normal(10, 5, n)

    #data for OLS
    y_OLS = 5 + 15*X+ eps
    
    #data for TSLS
    #交絡要因の影響は適当に入れている
    Z = cX[:,0]
    X = 5 + cX[:,1] + 0.3*conf
    y = 5 + 15*X + 0.4*conf + eps
    
    return y, X, Z, y_OLS


In [4]:

# ベクトルは行列計算出来ないので
# ndarrayに定数項用の列を追加
def add_const(X):
    r, c = X.shape
    nX = np.ones((r, c+1))
    nX[:,:-1] = X
    return nX

#OLSを解く
def solve_OLS(y, X):
    X = X[:, np.newaxis]
    X = add_const(X)
    ans = np.linalg.inv(X.T@X)@X.T@y

    return ans

In [ ]:
#普通にOLS解く

#test
df = pd.DataFrame(index=[], columns=["n", "OLS", "OLS with conf", "TSLS", "TSLS ML"])
#サンプルサイズを100から5000まで100ずつふやしながら予測してみる
i = 100
while i <= 3000:
    y,X,Z,y_OLS = create_sample(i)
    #ここ気持ち悪いことになってるごめん
    #X（shapeは(n,1)）の係数だけDFに入れている
    sr = pd.Series([i, solve_OLS(y_OLS, X)[0], solve_OLS(y, X)[0], solve_TSLS(y, X, Z)[0], solve_TSLS_ML(y, X, Z)[0]],index=["n", "OLS", "OLS with conf", "TSLS", "TSLS ML"])
    df = df.append(sr,ignore_index=True)
    print(i)
    i += 100

In [5]:
#ベイズでOLSを解く
import pymc3 as pm

#変数を格納するモデルオブジェクトを生成する
ols_model = pm.Model()

with ols_model:
    #モデルパラメータのpriorを設定
    beta = Normal('beta', mu=0, sd=10, shape=2)


In [ ]:
#勾配法でOLSを解く